In [20]:
from collections import Counter
import numpy as np 
import pandas as pd
import pickle

def process_data_for_labels(ticker):
    hm_days = 7
    df = pd.read_csv('sp500_joined_closes.csv', index_col=0)
    tickers = df.columns.values.tolist()
    df.fillna(0, inplace=True)
    
    for i in range(1, hm_days +1):
        df['{}_{}d'.format(ticker, i)] = (df[ticker].shift(-i) - df[ticker]) / df[ticker]
        
    df.fillna(0, inplace=True)
    return tickers, df

process_data_for_labels('AAPL')


def buy_sell_hold(*args):
    cols = [c for c in args]
    requirement = 0.02
    for col in cols:
        if col > requirement:
            return 1
        if col < -requirement:
            return -1
    return 0

def extract_features(ticker):
    tickers, df = process_data_for_labels(ticker)
    
    df['{}_target'.format(ticker)] = list(map(buy_sell_hold, 
                                              df['{}_1d'.format(ticker)],
                                              df['{}_2d'.format(ticker)],
                                              df['{}_3d'.format(ticker)],
                                              df['{}_4d'.format(ticker)],
                                              df['{}_5d'.format(ticker)],
                                              df['{}_6d'.format(ticker)],
                                              df['{}_7d'.format(ticker)]
                                             ))
    
    vals = df['{}_target'.format(ticker)].values.tolist()
    str_vals = [str(i) for i in vals]
    print('Data spread:', Counter(str_vals))
    df.fillna(0, inplace=True)
    
    df = df.replace([np.inf, -np.inf], np.nan)

    df_vals = df[[ticker for ticker in tickers]].pct_change()
    df_vals = df_vals.replace([np.inf, -np.inf], 0)
    df_vals.fillna(0, inplace=True)

    X = df_vals.values
    y = df['{}_target'.format(ticker)].values

    return X, y, df

extract_features('AAPL')

Data spread: Counter({'1': 2627, '-1': 2124, '0': 509})


(array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [-0.03973543, -0.01440926, -0.02857139, ...,  0.        ,
         -0.04842385,  0.        ],
        [ 0.02896544, -0.00292396, -0.00183843, ...,  0.        ,
         -0.00118335,  0.        ],
        ...,
        [ 0.00722993,  0.02550824, -0.01460896, ...,  0.00205547,
          0.03343781, -0.01119143],
        [ 0.01688307,  0.0081754 , -0.01519177, ...,  0.03309641,
          0.05813944, -0.01995862],
        [ 0.00022585, -0.02743285, -0.01115138, ..., -0.00926654,
         -0.02173893, -0.00540134]]),
 array([-1, -1, -1, ...,  0,  0,  0]),
                    MMM        AOS         ABT  ...   AAPL_6d   AAPL_7d  AAPL_target
 Date                                           ...                                 
 2000-01-03   25.863483   2.392432    9.262775  ... -0.171413 -0.221105           -1
 2000-01-04   24.835787   2.357959    8.998125  ... -0.149390 -0.056098        

KeyError: "None of [Index(['XOM_1d', 'XOM_2d', 'XOM_3d', 'XOM_4d', 'XOM_5d', 'XOM_6d', 'XOM_7d'], dtype='object')] are in the [columns]"

In [18]:
import collections
import numpy as np
import pandas as pd
import pickle

def process_data_for_labels(ticker):
    hm_days = 7
    df = pd.read_csv('sp500_joined_closes.csv', index_col=0)
    tickers = df.columns.values.tolist()
    df.fillna(0, inplace=True)
    
    for i in range(1, hm_days + 1):
        shift_column = '{}-{}d'.format(ticker, i)
        df[shift_column] = (df[ticker].shift(-i) - df[ticker]) / df[ticker]
        
    df.fillna(0, inplace=True)
    return tickers, df

def buy_sell_hold(cols, requirement=0.02):
    for col in cols:
        if col > requirement:
            return 1
        if col < -requirement:
            return -1
    return 0

def extract_features(ticker):
    tickers, df = process_data_for_labels(ticker)
    
    target_column = '{}_target'.format(ticker)
    df[target_column] = list(map(buy_sell_hold,
                                 df[[f'{ticker}_{i}d' for i in range(1, 8)]],
                                 requirement=0.02))
    
    vals = df[target_column].values.tolist()
    str_vals = [str(i) for i in vals]
    print('Data spread:', collections.Counter(str_vals))
    df.fillna(0, inplace=True)
    
    df = df.replace([np.inf, -np.inf], np.nan)

    df_vals = df[tickers].pct_change()
    df_vals = df_vals.replace([np.inf, -np.inf], 0)
    df_vals.fillna(0, inplace=True)

    X = df_vals.values
    y = df[target_column].values

    return X, y, df

extract_features('AAPL')


KeyError: "None of [Index(['AAPL_1d', 'AAPL_2d', 'AAPL_3d', 'AAPL_4d', 'AAPL_5d', 'AAPL_6d',\n       'AAPL_7d'],\n      dtype='object')] are in the [columns]"